JUST A QUICK REVISION OF CODE FROM !COURSES!.

In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler
from sklearn.linear_model import RANSACRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

df = pd.read_csv(r"housing.data", sep=" +", engine="python", header=None, names=["CRIM","ZN","INDUS","CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT","MEDV"])
df.tail(10)

In [ ]:
X = df.loc[:, "LSTAT"].values.reshape(-1,1)
X

In [ ]:
y = df["MEDV"].values
y

In [ ]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.14)

In [ ]:
ransac = RANSACRegressor(residual_threshold=20).fit(X_train, y_train)
r2_ransac = r2_score(y_test, ransac.predict(X_test))
print(f"RANSAC lin - reg model R2 value = {r2_ransac}")

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
r2_lin_reg = r2_score(y_test, lin_reg.predict(X_test))
print(f"Lin - Reg model R2 value = {r2_lin_reg}")

Classic Lin Reg model got better result. Aiming for threshold that will outmatch RANSAC as better one.

In [ ]:
best_r2 = 0
for i in range(8,37):
    tmp_ransac = RANSACRegressor(residual_threshold=i)
    tmp_ransac.fit(X_train, y_train)
    tmp_r2 = r2_score(y_test, tmp_ransac.predict(X_test))
    print(f"Treshold {i} - R2 {tmp_r2}")
    if tmp_r2 > best_r2:
        the_best_r2 = tmp_r2
        ransac = tmp_ransac

In [ ]:
line_X = np.arange(X.min(), X.max())[:, np.newaxis]
line_y_ransac = ransac.predict(line_X)
line_y_lr = lin_reg.predict(line_X)

In [ ]:
plt.scatter(X_train[ransac.inlier_mask_], y_train[ransac.inlier_mask_], color='green', marker='.',
            label='Inliers')
plt.scatter(X_train[~ransac.inlier_mask_], y_train[~ransac.inlier_mask_], color='gold', marker='.',
            label='Outliers')
plt.plot(line_X, line_y_ransac, color='red', linewidth=2, label='RANSAC regressor')
plt.plot(line_X, line_y_lr, color='blue', linewidth=2, label='Linear regression')
plt.legend(loc='lower right')
plt.xlabel("Input")
plt.ylabel("Response")
plt.show()

In [ ]:
r2_ransac = r2_score(y_test, ransac.predict(X_test))
print("Ransac regression result: {}".format(r2_ransac))
print("Linear regression result: {}".format(r2_lin_reg))